In [112]:
from aligment import align_image_dnn
from keras.models import load_model
from transform import four_point_transform
import cv2
import numpy as np
from scipy import ndimage
from PIL import Image

model = load_model("aligment_model.h5", compile=False)
ZERO_IMAGE =  np.empty((0,0,3))
SHOW_IMAGE = True

def four_point_transform_with_mask(mask, orin, box):
    warped_mask = four_point_transform(mask, box)
    mask_shape = mask.shape
    orin_shape = orin.shape
    w, h = mask_shape[0:2]
    W, H = orin_shape[0:2]
    rh = H / h
    rw = W / w
    BOX = np.zeros_like(box)
    BOX[:, 0] = box[:, 0] * rh
    BOX[:, 1] = box[:, 1] * rw
    BOX = BOX.astype(np.int16)
    warped_orin = four_point_transform(orin, BOX)
    return warped_mask, warped_orin

def show(np_img, show=SHOW_IMAGE):
    if show:
        Image.fromarray(np_img).show()


In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img = cv.imread('out/bounding.jpg',0)
# global thresholding
ret1,th1 = cv.threshold(img,15,255,cv.THRESH_BINARY)
# Otsu's thresholding
ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# Otsu's thresholding after Gaussian filtering
blur = cv.GaussianBlur(img,(5,5),0)
ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]
plt.figure(figsize=(20,20))
for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

In [113]:
def find_corner(crop):
    h = crop.shape[0]
    w = crop.shape[1]
    x_pos = np.zeros(h)
    for i in range(h):
        for j in range(w):
            if j == w-1:
                x_pos[i]=j
            if crop[i, j]>20:
                x_pos[i]=j
                break
        
    sum_dis = np.arange(h)+x_pos
    argmin = np.argmin(sum_dis)
    corner = (int(x_pos[argmin]), argmin)
    return corner

def transform_with_up_bottom_points(mask, up_points, bottom_points):
    h = mask.shape[0]
    straighted = np.zeros_like(mask)
    for i in range(len(up_points)-1):
        _w = int(up_points[i+1][0] - up_points[i][0])
        rect = np.array([up_points[i], up_points[i+1], bottom_points[i+1], bottom_points[i]] , dtype="float32")
        dst = np.array([[0, 0], [_w, 0],  [_w, h], [0, h]] , dtype="float32")
        M = cv2.getPerspectiveTransform(rect, dst)
        warped_i = cv2.warpPerspective(mask, M, (_w, h))
        straighted[:, int(up_points[i][0]):int(up_points[i+1][0])] = warped_i
    return straighted

def straighten(warped, warped_origin, corners, factor=5):
    straighted = np.zeros_like(warped)
    straighted_origin = np.zeros_like(warped_origin)
    h, w = warped.shape[:2]
    H, W = warped_origin.shape[:2]
    h_r, w_r  = H/h, W/w
    up_points = [corners[0]]
    bottom_points = [corners[3]]
    UP_points = [(int(corners[0][0]*w_r), int(corners[0][1]*h_r))]
    BOTTOM_points = [(int(corners[3][0]*w_r), int(corners[3][1]*h_r))]

    for i in range(1, factor):
        up_point = None
        bottom_point = None
        for j in range(h//5):
            if warped[j, int(w/factor*i)]>100 :
                up_point = [int(w/factor*i), j]
                break

        for j in range(h//5):
            if warped[h-j-1, int(w/factor*i)]>100 :
                bottom_point = [int(w/factor*i), h-j-1]
                break

        if up_point is not None and bottom_point is not None:
            up_points.append(up_point)
            UP_points.append((int(up_point[0]*w_r), int(up_point[1]*h_r)))
            bottom_points.append(bottom_point)
            BOTTOM_points.append((int(bottom_point[0]*w_r), int(bottom_point[1]*h_r)))

    up_points.append(corners[1])
    bottom_points.append(corners[2])        
    UP_points.append((int(corners[1][0]*w_r), int(corners[1][1]*h_r)))
    BOTTOM_points.append((int(corners[2][0]*w_r), int(corners[2][1]*h_r)))

    warped = cv2.cvtColor(warped, cv2.COLOR_GRAY2BGR)
    for point in up_points:
        cv2.circle(warped, tuple(point), 3, (255, 0, 255), -1)
    for point in bottom_points:
        cv2.circle(warped, tuple(point), 3, (255, 0, 255), -1)

    straighted= transform_with_up_bottom_points(warped, up_points, bottom_points)
    cv2.imwrite("out/straighted.jpg", straighted)
    straighted_orin= transform_with_up_bottom_points(warped_origin, UP_points, BOTTOM_points)
    return straighted_orin


def align_image(img):
    cv2.imwrite("out/img.jpg", img)
    HEIGHT = 512
    WIDTH = 512
    resized_img = cv2.resize(img, (HEIGHT, WIDTH))
    bounding = model.predict(np.array([resized_img / 255.0]))[0]
    bounding = (255 * (bounding + 1) / 2).astype(np.uint8)
    cv2.imwrite("out/bounding.jpg", bounding)
    kernel = np.ones((3, 3), np.uint8)
#     bounding = cv2.erode(bounding, kernel, iterations=1)
#     blur = cv2.GaussianBlur(bounding,(5,5),0)
#     cv2.imwrite("out/blur.jpg", blur)
#     ret3, Thres = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     Thres = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
#             cv2.THRESH_BINARY,31, -15)
    Thres = cv2.adaptiveThreshold(bounding,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY,31, -15)
    Thres = cv2.GaussianBlur(Thres,(5,5),0)
    Thres = cv2.erode(Thres, kernel, iterations=1)
    Thres2 = Thres.copy()
#     cv2.imwrite("out/adap1.jpg", th2)
#     cv2.imwrite("out/adap2.jpg", th3)
    cv2.imwrite("out/Thres.jpg", Thres)
    
    contours,hierarchy = cv2.findContours(Thres,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt  in contours:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        im = cv2.drawContours(Thres2,[box],0,(255,255,255), -1)
    cv2.imwrite("out/Thres2.jpg", Thres2)
    contours,hierarchy = cv2.findContours(Thres2,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    areas = np.zeros(len(contours))
    for i  in range(len(contours)):
        areas[i] = cv2.contourArea(contours[i])
    sorted_area = np.sort(areas)
    biggest_cnt = contours[np.argmax(areas)]
    rect = cv2.minAreaRect(biggest_cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)



    warped_mask, warped_origin = four_point_transform_with_mask(Thres, img, box)
    cv2.imwrite("out/warped_mask.jpg", warped_mask)
    cv2.imwrite("out/warped_origin.jpg", warped_origin)
    warped_mask_draw = cv2.cvtColor(warped_mask, cv2.COLOR_GRAY2BGR)
    kernel = np.ones((3, 3), np.uint8)
    warped_mask_erode = cv2.erode(warped_mask, kernel, iterations=1)
    h, w = warped_mask.shape[0:2]
    pad = 4
    left_top_crop = warped_mask_erode[:h//pad, :w//pad]

    right_top_crop = warped_mask_erode[:h//pad, -w//pad:]
    right_top_crop_flip = cv2.flip(right_top_crop, 1)

    left_bottom_crop = warped_mask_erode[-h//pad:, :w//pad]
    left_bottom_crop_flip = cv2.flip(left_bottom_crop, 0)

    right_bottom_crop = warped_mask_erode[-h//pad:, -w//pad:]
    right_bottom_crop_flip = cv2.flip(right_bottom_crop, -1)

    left_top_corner = find_corner(left_top_crop)
    right_top_corner_flip = find_corner(right_top_crop_flip)
    left_bottom_corner_flip = find_corner(left_bottom_crop_flip)
    right_bottom_corner_flip = find_corner(right_bottom_crop_flip)

    right_top_corner = (w-right_top_corner_flip[0], right_top_corner_flip[1])
    left_bottom_corner = (left_bottom_corner_flip[0],h- left_bottom_corner_flip[1])
    right_bottom_corner = (w- right_bottom_corner_flip[0],h- right_bottom_corner_flip[1])
    cv2.circle(warped_mask_draw, left_top_corner, 3, (255, 0, 255), -1)
    cv2.circle(warped_mask_draw, right_top_corner, 3, (255, 0, 255), -1)
    cv2.circle(warped_mask_draw, left_bottom_corner, 3, (255, 0, 255), -1)
    cv2.circle(warped_mask_draw, right_bottom_corner, 3, (255, 0, 255), -1)
    h, w = warped_mask_draw.shape[:2]
    H, W = warped_origin.shape[:2]
    h_r, w_r  = H/h, W/w

    rect =  np.array([list(left_top_corner), list(right_top_corner), list(right_bottom_corner), list(left_bottom_corner)], dtype = "float32")
    dst =  np.array([[0, left_top_corner[1]], [w, right_top_corner[1]], [w, right_bottom_corner[1]], [0, left_bottom_corner[1]]], dtype = "float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(warped_mask_draw, M, (w, h))

    RECT = np.zeros_like(rect)
    DST = np.zeros_like(dst)

    RECT[:, 0] = rect[:, 0]*w_r
    RECT[:, 1] = rect[:, 1]*h_r

    DST[:, 0] = dst[:, 0]*w_r
    DST[:, 1] = dst[:, 1]*h_r

    M2 = cv2.getPerspectiveTransform(RECT, DST)
    warped_origin2 = cv2.warpPerspective(warped_origin, M2, (W, H))

    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("out/warped.jpg", warped)
    factor = 5
    if W/H>1.2:
        factor = 7
    if W/H<0.8:
        factor = 3
    straighted_orin = straighten(warped.copy(),warped_origin2, dst, factor)
    t = int(straighted_orin.shape[0]*0.01)
    straighted_orin = straighted_orin[t:]
    cv2.imwrite("out/straighted_orin.jpg", straighted_orin)
    return straighted_orin

In [119]:
img = cv2.imread("samples/11.jpg")
show(img)
straighted_orin = align_image(img)
show(straighted_orin)